# 4.4 Train and Evaluate Boosted Models

## Introduction

In the previous notebooks, we built gradient boosting models using XGBoost, LightGBM, and CatBoost. Now we focus on **training best practices** and **comprehensive evaluation**. Proper training techniques like early stopping prevent overfitting, while thorough evaluation ensures our models will perform well on new student data.

We also introduce **SHAP (SHapley Additive exPlanations)**, a powerful method for interpreting model predictions that provides both global feature importance and local explanations for individual students.

### Learning Objectives

By the end of this notebook, you will be able to:

1. Implement early stopping to prevent overfitting in gradient boosting models
2. Perform cross-validation with early stopping for robust model evaluation
3. Evaluate models using multiple classification metrics
4. Interpret model predictions using SHAP values
5. Explain individual predictions to stakeholders

## 1. Setup and Data Preparation

### 1.1 Import Libraries

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Scikit-learn
from sklearn.model_selection import (
    train_test_split, cross_val_score, StratifiedKFold, cross_validate
)
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, precision_recall_curve, average_precision_score,
    confusion_matrix, classification_report, brier_score_loss
)
from sklearn.calibration import calibration_curve

# Gradient Boosting Libraries
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# SHAP for model interpretation
import shap

print("Libraries loaded successfully!")

### 1.2 Load and Prepare Data

In [ ]:
# Generate synthetic student data
np.random.seed(42)
n_students = 3000

data = {
    'STUDENT_ID': range(1, n_students + 1),
    'HS_GPA': np.random.normal(3.2, 0.5, n_students).clip(2.0, 4.0),
    'MATH_PLACEMENT': np.random.choice(['Remedial', 'College-Ready', 'Advanced'], n_students, p=[0.2, 0.5, 0.3]),
    'FIRST_GEN': np.random.choice(['Yes', 'No'], n_students, p=[0.35, 0.65]),
    'PELL_ELIGIBLE': np.random.choice(['Yes', 'No'], n_students, p=[0.40, 0.60]),
    'RESIDENCY': np.random.choice(['In-State', 'Out-of-State', 'International'], n_students, p=[0.7, 0.2, 0.1]),
    'UNITS_ATTEMPT_1': np.random.normal(14, 2, n_students).clip(6, 18).astype(int),
    'GPA_1': np.random.normal(2.8, 0.7, n_students).clip(0.0, 4.0),
    'DFW_RATE_1': np.random.beta(2, 8, n_students),
    'UNITS_ATTEMPT_2': np.random.normal(14, 2, n_students).clip(6, 18).astype(int),
    'GPA_2': np.random.normal(2.9, 0.6, n_students).clip(0.0, 4.0),
    'DFW_RATE_2': np.random.beta(2, 8, n_students),
}

df = pd.DataFrame(data)

# Derived features
df['CUM_GPA'] = (df['GPA_1'] + df['GPA_2']) / 2
df['CUM_UNITS'] = df['UNITS_ATTEMPT_1'] + df['UNITS_ATTEMPT_2']
df['AVG_DFW'] = (df['DFW_RATE_1'] + df['DFW_RATE_2']) / 2
df['GPA_TREND'] = df['GPA_2'] - df['GPA_1']  # Improvement or decline

# Generate target
departure_prob = (
    0.3 - 0.15 * (df['CUM_GPA'] - 2.5) + 0.3 * df['AVG_DFW']
    + 0.05 * (df['FIRST_GEN'] == 'Yes') - 0.02 * (df['HS_GPA'] - 3.0)
    + 0.05 * (df['MATH_PLACEMENT'] == 'Remedial')
    - 0.05 * df['GPA_TREND']  # Improvement reduces risk
)
departure_prob = departure_prob.clip(0.05, 0.95)
df['DEPARTED'] = np.random.binomial(1, departure_prob)

# Define columns
categorical_cols = ['MATH_PLACEMENT', 'FIRST_GEN', 'PELL_ELIGIBLE', 'RESIDENCY']
numerical_cols = ['HS_GPA', 'UNITS_ATTEMPT_1', 'GPA_1', 'DFW_RATE_1', 
                  'UNITS_ATTEMPT_2', 'GPA_2', 'DFW_RATE_2', 
                  'CUM_GPA', 'CUM_UNITS', 'AVG_DFW', 'GPA_TREND']

feature_cols = categorical_cols + numerical_cols

print(f"Dataset: {n_students} students")
print(f"Departure rate: {df['DEPARTED'].mean():.1%}")

### 1.3 Create Validation Set

For early stopping, we need a separate validation set to monitor performance during training.

In [ ]:
# Split: 60% train, 20% validation, 20% test
X = df[feature_cols]
y = df['DEPARTED']

# First split: 80% train+val, 20% test
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Second split: 75% train, 25% val (of the 80%)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, random_state=42, stratify=y_trainval
)

print(f"Training set:   {len(X_train)} students ({len(X_train)/len(X):.0%})")
print(f"Validation set: {len(X_val)} students ({len(X_val)/len(X):.0%})")
print(f"Test set:       {len(X_test)} students ({len(X_test)/len(X):.0%})")
print(f"\nDeparture rates:")
print(f"  Train: {y_train.mean():.1%}")
print(f"  Val:   {y_val.mean():.1%}")
print(f"  Test:  {y_test.mean():.1%}")

In [ ]:
# Prepare encoded versions for XGBoost and LightGBM

# One-hot encode for XGBoost
X_train_xgb = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_val_xgb = pd.get_dummies(X_val, columns=categorical_cols, drop_first=True)
X_test_xgb = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# Ensure consistent columns
X_val_xgb = X_val_xgb.reindex(columns=X_train_xgb.columns, fill_value=0)
X_test_xgb = X_test_xgb.reindex(columns=X_train_xgb.columns, fill_value=0)

# Label encode for LightGBM
X_train_lgb = X_train.copy()
X_val_lgb = X_val.copy()
X_test_lgb = X_test.copy()

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X_train_lgb[col] = le.fit_transform(X_train_lgb[col])
    X_val_lgb[col] = le.transform(X_val_lgb[col])
    X_test_lgb[col] = le.transform(X_test_lgb[col])
    label_encoders[col] = le

print("Data prepared for all three libraries!")

## 2. Early Stopping

### 2.1 Why Early Stopping?

**Early stopping** monitors performance on a validation set during training and stops when performance stops improving. This prevents overfitting and saves training time.

Without early stopping:
- Model may train for more iterations than necessary
- Training performance keeps improving
- Validation performance peaks then degrades (overfitting)

With early stopping:
- Training stops when validation performance stops improving
- Automatically finds optimal number of iterations
- Acts as regularization

In [ ]:
# Demonstrate overfitting without early stopping
np.random.seed(42)

# Simulate training and validation loss over iterations
n_iters = 200
iterations = np.arange(1, n_iters + 1)

# Training loss decreases continuously
train_loss = 0.7 * np.exp(-0.02 * iterations) + 0.05 + np.random.normal(0, 0.01, n_iters)

# Validation loss decreases then increases (overfitting)
val_loss = 0.7 * np.exp(-0.02 * iterations) + 0.1 + 0.0005 * (iterations - 80) ** 2 / 100
val_loss[:80] = 0.7 * np.exp(-0.02 * iterations[:80]) + 0.1
val_loss = val_loss + np.random.normal(0, 0.015, n_iters)

# Find best iteration
best_iter = np.argmin(val_loss) + 1

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=iterations, y=train_loss,
    mode='lines', name='Training Loss',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=iterations, y=val_loss,
    mode='lines', name='Validation Loss',
    line=dict(color='red', width=2)
))

fig.add_vline(x=best_iter, line_dash="dash", line_color="green",
              annotation_text=f"Best: {best_iter}")

# Highlight overfitting region
fig.add_vrect(x0=best_iter, x1=n_iters, fillcolor="red", opacity=0.1,
              annotation_text="Overfitting", annotation_position="top right")

fig.update_layout(
    title='Early Stopping: Preventing Overfitting',
    xaxis_title='Boosting Iterations',
    yaxis_title='Loss',
    height=450
)

fig.show()

print(f"Optimal stopping point: {best_iter} iterations")
print(f"Without early stopping: {n_iters} iterations (wasted {n_iters - best_iter} iterations)")

### 2.2 Early Stopping in XGBoost

In [ ]:
# XGBoost with early stopping
xgb_model = XGBClassifier(
    n_estimators=500,          # Set high, early stopping will find optimal
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    eval_metric='logloss',
    early_stopping_rounds=20,  # Stop if no improvement for 20 rounds
    use_label_encoder=False
)

# Train with validation set
xgb_model.fit(
    X_train_xgb, y_train,
    eval_set=[(X_train_xgb, y_train), (X_val_xgb, y_val)],
    verbose=False
)

print("XGBoost Training with Early Stopping")
print("=" * 50)
print(f"Requested iterations: 500")
print(f"Best iteration: {xgb_model.best_iteration}")
print(f"Best validation score: {xgb_model.best_score:.4f}")

In [ ]:
# Plot training history
results = xgb_model.evals_result()

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=results['validation_0']['logloss'],
    mode='lines', name='Training',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    y=results['validation_1']['logloss'],
    mode='lines', name='Validation',
    line=dict(color='red', width=2)
))

fig.add_vline(x=xgb_model.best_iteration, line_dash="dash", line_color="green",
              annotation_text=f"Best: {xgb_model.best_iteration}")

fig.update_layout(
    title='XGBoost Training History with Early Stopping',
    xaxis_title='Iteration',
    yaxis_title='Log Loss',
    height=400
)

fig.show()

### 2.3 Early Stopping in LightGBM

In [ ]:
# LightGBM with early stopping
lgb_model = LGBMClassifier(
    n_estimators=500,
    num_leaves=31,
    learning_rate=0.1,
    random_state=42,
    verbose=-1
)

# Train with callbacks for early stopping
lgb_model.fit(
    X_train_lgb, y_train,
    eval_set=[(X_val_lgb, y_val)],
    eval_metric='logloss',
    callbacks=[
        lgb.early_stopping(stopping_rounds=20, verbose=False),
        lgb.log_evaluation(period=0)  # Suppress logging
    ],
    categorical_feature=categorical_cols
)

print("LightGBM Training with Early Stopping")
print("=" * 50)
print(f"Requested iterations: 500")
print(f"Best iteration: {lgb_model.best_iteration_}")
print(f"Best validation score: {lgb_model.best_score_['valid_0']['binary_logloss']:.4f}")

### 2.4 Early Stopping in CatBoost

In [ ]:
# CatBoost with early stopping (built-in)
cat_model = CatBoostClassifier(
    iterations=500,
    depth=6,
    learning_rate=0.1,
    cat_features=categorical_cols,
    early_stopping_rounds=20,
    random_state=42,
    verbose=0
)

# Train with validation set
cat_model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    use_best_model=True
)

print("CatBoost Training with Early Stopping")
print("=" * 50)
print(f"Requested iterations: 500")
print(f"Best iteration: {cat_model.best_iteration_}")

In [ ]:
# Compare best iterations across models
comparison = pd.DataFrame({
    'Model': ['XGBoost', 'LightGBM', 'CatBoost'],
    'Requested': [500, 500, 500],
    'Best Iteration': [xgb_model.best_iteration, lgb_model.best_iteration_, cat_model.best_iteration_],
})
comparison['Savings'] = comparison['Requested'] - comparison['Best Iteration']
comparison['Savings %'] = (comparison['Savings'] / comparison['Requested'] * 100).round(1)

print("Early Stopping Comparison")
comparison

## 3. Cross-Validation Strategies

### 3.1 Stratified K-Fold Cross-Validation

For imbalanced classes like student departure, **stratified** K-fold ensures each fold has the same class distribution as the full dataset.

In [ ]:
# Stratified 5-Fold CV on the full training+validation set
X_full = pd.concat([X_train, X_val])
y_full = pd.concat([y_train, y_val])

# Prepare for CatBoost (easiest to use)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validate CatBoost
cv_scores = cross_validate(
    CatBoostClassifier(
        iterations=200,
        depth=6,
        learning_rate=0.1,
        cat_features=categorical_cols,
        random_state=42,
        verbose=0
    ),
    X_full, y_full,
    cv=cv,
    scoring=['accuracy', 'roc_auc', 'f1', 'precision', 'recall'],
    return_train_score=True
)

print("5-Fold Stratified Cross-Validation Results")
print("=" * 60)
print(f"{'Metric':<15} {'Train Mean':>12} {'Test Mean':>12} {'Test Std':>12}")
print("-" * 60)

for metric in ['accuracy', 'roc_auc', 'f1', 'precision', 'recall']:
    train_scores = cv_scores[f'train_{metric}']
    test_scores = cv_scores[f'test_{metric}']
    print(f"{metric:<15} {train_scores.mean():>12.3f} {test_scores.mean():>12.3f} {test_scores.std():>12.3f}")

### 3.2 Cross-Validation with Early Stopping

For production models, combine cross-validation with early stopping to get both robust estimates and optimal iteration counts.

In [ ]:
# Manual CV with early stopping
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_results = []
best_iterations = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_full, y_full)):
    X_cv_train = X_full.iloc[train_idx]
    X_cv_val = X_full.iloc[val_idx]
    y_cv_train = y_full.iloc[train_idx]
    y_cv_val = y_full.iloc[val_idx]
    
    # Train with early stopping
    model = CatBoostClassifier(
        iterations=500,
        depth=6,
        learning_rate=0.1,
        cat_features=categorical_cols,
        early_stopping_rounds=20,
        random_state=42,
        verbose=0
    )
    
    model.fit(
        X_cv_train, y_cv_train,
        eval_set=(X_cv_val, y_cv_val),
        use_best_model=True
    )
    
    # Evaluate
    y_pred_proba = model.predict_proba(X_cv_val)[:, 1]
    y_pred = model.predict(X_cv_val)
    
    cv_results.append({
        'Fold': fold + 1,
        'Best Iteration': model.best_iteration_,
        'ROC-AUC': roc_auc_score(y_cv_val, y_pred_proba),
        'Accuracy': accuracy_score(y_cv_val, y_pred),
        'F1': f1_score(y_cv_val, y_pred)
    })
    best_iterations.append(model.best_iteration_)

cv_results_df = pd.DataFrame(cv_results)

print("Cross-Validation with Early Stopping")
print("=" * 60)
print(cv_results_df.round(4))
print("\nSummary:")
print(f"  Mean Best Iteration: {np.mean(best_iterations):.0f} (std: {np.std(best_iterations):.0f})")
print(f"  Mean ROC-AUC: {cv_results_df['ROC-AUC'].mean():.4f} (std: {cv_results_df['ROC-AUC'].std():.4f})")

## 4. Comprehensive Model Evaluation

### 4.1 Classification Metrics

In [ ]:
# Final evaluation on test set using CatBoost
# Train on full train+val with optimal iterations
optimal_iters = int(np.mean(best_iterations))

final_model = CatBoostClassifier(
    iterations=optimal_iters,
    depth=6,
    learning_rate=0.1,
    cat_features=categorical_cols,
    random_state=42,
    verbose=0
)

final_model.fit(X_full, y_full)

# Predict on test set
y_pred = final_model.predict(X_test)
y_pred_proba = final_model.predict_proba(X_test)[:, 1]

print("Final Model Performance on Test Set")
print("=" * 50)
print(f"Training iterations: {optimal_iters}")
print()
print(classification_report(y_test, y_pred, target_names=['Retained', 'Departed']))

In [ ]:
# Confusion matrix visualization
cm = confusion_matrix(y_test, y_pred)

fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=['Predicted Retained', 'Predicted Departed'],
    y=['Actual Retained', 'Actual Departed'],
    text=[[f'{cm[i,j]}<br>({cm[i,j]/cm.sum()*100:.1f}%)' for j in range(2)] for i in range(2)],
    texttemplate='%{text}',
    textfont=dict(size=14),
    colorscale='Blues',
    showscale=True
))

fig.update_layout(
    title='Confusion Matrix on Test Set',
    xaxis_title='Predicted',
    yaxis_title='Actual',
    height=400
)

fig.show()

### 4.2 ROC and Precision-Recall Curves

In [ ]:
# Calculate curves
fpr, tpr, roc_thresholds = roc_curve(y_test, y_pred_proba)
precision, recall, pr_thresholds = precision_recall_curve(y_test, y_pred_proba)

roc_auc = roc_auc_score(y_test, y_pred_proba)
avg_precision = average_precision_score(y_test, y_pred_proba)

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=(
    f'ROC Curve (AUC = {roc_auc:.3f})',
    f'Precision-Recall Curve (AP = {avg_precision:.3f})'
))

# ROC Curve
fig.add_trace(go.Scatter(
    x=fpr, y=tpr, mode='lines',
    name='ROC', line=dict(color='blue', width=2)
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1], mode='lines',
    name='Random', line=dict(color='gray', dash='dash')
), row=1, col=1)

# Precision-Recall Curve
fig.add_trace(go.Scatter(
    x=recall, y=precision, mode='lines',
    name='PR', line=dict(color='green', width=2)
), row=1, col=2)

# Baseline (random classifier)
baseline = y_test.mean()
fig.add_hline(y=baseline, line_dash="dash", line_color="gray", row=1, col=2)

fig.update_xaxes(title_text='False Positive Rate', row=1, col=1)
fig.update_yaxes(title_text='True Positive Rate', row=1, col=1)
fig.update_xaxes(title_text='Recall', row=1, col=2)
fig.update_yaxes(title_text='Precision', row=1, col=2)

fig.update_layout(height=400, showlegend=False,
                  title_text='Model Discrimination Performance')
fig.show()

### 4.3 Calibration Curves

A **calibration curve** shows whether predicted probabilities match actual outcomes. Well-calibrated models are essential for risk scoring.

In [ ]:
# Calculate calibration curve
prob_true, prob_pred = calibration_curve(y_test, y_pred_proba, n_bins=10)

# Calculate Brier score (lower is better)
brier = brier_score_loss(y_test, y_pred_proba)

fig = go.Figure()

# Perfect calibration line
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1], mode='lines',
    name='Perfect Calibration',
    line=dict(color='gray', dash='dash')
))

# Model calibration
fig.add_trace(go.Scatter(
    x=prob_pred, y=prob_true, mode='lines+markers',
    name='Model Calibration',
    line=dict(color='blue', width=2),
    marker=dict(size=10)
))

fig.update_layout(
    title=f'Calibration Curve (Brier Score = {brier:.4f})',
    xaxis_title='Mean Predicted Probability',
    yaxis_title='Fraction of Positives',
    height=450,
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1])
)

fig.show()

print("Calibration Interpretation:")
print("- Perfect calibration: points on diagonal")
print("- Above diagonal: model underestimates probability")
print("- Below diagonal: model overestimates probability")
print(f"\nBrier Score: {brier:.4f} (0 = perfect, 0.25 = random)")

## 5. SHAP for Model Interpretation

### 5.1 Introduction to SHAP

**SHAP (SHapley Additive exPlanations)** uses game theory to explain individual predictions:

- Each feature's contribution to a prediction is its "Shapley value"
- Shapley values sum to the difference between the prediction and the baseline
- Provides both **global** (feature importance) and **local** (individual) explanations

SHAP is particularly valuable in higher education because it helps explain **why** a student is flagged as at-risk.

In [ ]:
# Create SHAP explainer
# CatBoost has native SHAP support
explainer = shap.TreeExplainer(final_model)

# Calculate SHAP values for test set
# Use a sample for faster computation
sample_size = min(500, len(X_test))
X_sample = X_test.head(sample_size)

shap_values = explainer.shap_values(X_sample)

print(f"SHAP values calculated for {sample_size} students")
print(f"Shape of SHAP values: {shap_values.shape}")

### 5.2 SHAP Summary Plot

The summary plot shows global feature importance and how features affect predictions.

In [ ]:
# Calculate mean absolute SHAP values for global importance
mean_abs_shap = np.abs(shap_values).mean(axis=0)
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Mean |SHAP|': mean_abs_shap
}).sort_values('Mean |SHAP|', ascending=True)

# Plot
fig = px.bar(
    feature_importance.tail(15),
    x='Mean |SHAP|', y='Feature',
    orientation='h',
    title='SHAP Feature Importance (Mean Absolute SHAP Value)',
    color='Mean |SHAP|',
    color_continuous_scale='Blues'
)

fig.update_layout(height=500, yaxis_title='', xaxis_title='Mean |SHAP Value|')
fig.show()

In [ ]:
# Create a beeswarm-style SHAP plot using Plotly
# Get top 10 features
top_features = feature_importance.tail(10)['Feature'].tolist()

# Prepare data for visualization
plot_data = []
for i, feat in enumerate(top_features):
    feat_idx = feature_cols.index(feat)
    feat_shap = shap_values[:, feat_idx]
    feat_values = X_sample[feat].values
    
    # Normalize feature values for coloring
    if X_sample[feat].dtype in ['object', 'category']:
        feat_norm = pd.factorize(feat_values)[0]
    else:
        feat_norm = (feat_values - feat_values.min()) / (feat_values.max() - feat_values.min() + 1e-10)
    
    for j in range(len(feat_shap)):
        plot_data.append({
            'Feature': feat,
            'SHAP Value': feat_shap[j],
            'Feature Value (normalized)': feat_norm[j],
            'Feature Index': i + np.random.normal(0, 0.1)  # Add jitter
        })

plot_df = pd.DataFrame(plot_data)

fig = px.scatter(
    plot_df, x='SHAP Value', y='Feature',
    color='Feature Value (normalized)',
    color_continuous_scale='RdBu_r',
    title='SHAP Summary Plot: Feature Impact on Departure Prediction',
    opacity=0.6
)

fig.add_vline(x=0, line_dash="dash", line_color="gray")
fig.update_layout(height=500, xaxis_title='SHAP Value (impact on departure probability)')
fig.update_coloraxes(colorbar_title='Feature<br>Value')

fig.show()

print("Interpretation:")
print("- Each dot is one student")
print("- Red = high feature value, Blue = low feature value")
print("- Right of center = increases departure probability")
print("- Left of center = decreases departure probability")

### 5.3 SHAP Dependence Plots

Dependence plots show how a feature's value affects its SHAP value.

In [ ]:
# SHAP dependence plot for CUM_GPA
feat = 'CUM_GPA'
feat_idx = feature_cols.index(feat)

fig = px.scatter(
    x=X_sample[feat],
    y=shap_values[:, feat_idx],
    color=X_sample['AVG_DFW'],  # Color by interacting feature
    color_continuous_scale='RdBu_r',
    title=f'SHAP Dependence Plot: {feat}',
    labels={'x': feat, 'y': 'SHAP Value', 'color': 'AVG_DFW'},
    opacity=0.7
)

fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.update_layout(height=450)
fig.show()

print("Interpretation:")
print("- Higher GPA generally has negative SHAP values (reduces departure risk)")
print("- Color shows interaction: students with high DFW rates (red) have different patterns")

In [ ]:
# Multiple dependence plots
features_to_plot = ['CUM_GPA', 'AVG_DFW', 'HS_GPA', 'GPA_TREND']

fig = make_subplots(rows=2, cols=2, subplot_titles=features_to_plot)

for i, feat in enumerate(features_to_plot):
    row = i // 2 + 1
    col = i % 2 + 1
    feat_idx = feature_cols.index(feat)
    
    fig.add_trace(go.Scatter(
        x=X_sample[feat],
        y=shap_values[:, feat_idx],
        mode='markers',
        marker=dict(color='blue', size=5, opacity=0.5),
        name=feat,
        showlegend=False
    ), row=row, col=col)
    
    fig.add_hline(y=0, line_dash="dash", line_color="gray", row=row, col=col)

fig.update_layout(height=600, title_text='SHAP Dependence Plots for Key Features')
fig.show()

### 5.4 Individual Prediction Explanations

SHAP allows us to explain why a specific student was predicted to be at risk.

In [ ]:
# Pick a high-risk student to explain
high_risk_idx = y_pred_proba.argmax()
high_risk_student = X_test.iloc[[high_risk_idx]]
high_risk_prob = y_pred_proba[high_risk_idx]

# Calculate SHAP values for this student
individual_shap = explainer.shap_values(high_risk_student)[0]

print(f"High-Risk Student Analysis")
print("=" * 50)
print(f"Predicted Departure Probability: {high_risk_prob:.1%}")
print(f"Actual Outcome: {'Departed' if y_test.iloc[high_risk_idx] == 1 else 'Retained'}")
print("\nStudent Profile:")
for col in feature_cols:
    print(f"  {col}: {high_risk_student[col].values[0]}")

In [ ]:
# Waterfall plot for individual explanation
base_value = explainer.expected_value

# Sort features by absolute SHAP value
shap_df = pd.DataFrame({
    'Feature': feature_cols,
    'Value': [high_risk_student[f].values[0] for f in feature_cols],
    'SHAP': individual_shap
}).sort_values('SHAP', key=abs, ascending=True)

# Take top 10 features
top_shap = shap_df.tail(10)

# Create waterfall-style plot
colors = ['red' if x > 0 else 'blue' for x in top_shap['SHAP']]

fig = go.Figure()

fig.add_trace(go.Bar(
    y=[f"{row['Feature']} = {row['Value']}" for _, row in top_shap.iterrows()],
    x=top_shap['SHAP'],
    orientation='h',
    marker_color=colors
))

fig.add_vline(x=0, line_dash="solid", line_color="gray")

fig.update_layout(
    title=f'SHAP Explanation for High-Risk Student (P(Departed) = {high_risk_prob:.1%})',
    xaxis_title='SHAP Value (impact on log-odds)',
    yaxis_title='',
    height=450,
    annotations=[
        dict(x=0.95, y=0.05, xref='paper', yref='paper',
             text=f'Base rate: {base_value:.2f}', showarrow=False,
             font=dict(size=10))
    ]
)

fig.show()

print("\nInterpretation:")
print("- Red bars push prediction toward 'Departed'")
print("- Blue bars push prediction toward 'Retained'")
print("- Length indicates strength of impact")

In [ ]:
# Compare high-risk vs low-risk student
low_risk_idx = y_pred_proba.argmin()
low_risk_student = X_test.iloc[[low_risk_idx]]
low_risk_prob = y_pred_proba[low_risk_idx]
low_risk_shap = explainer.shap_values(low_risk_student)[0]

# Create comparison
comparison = pd.DataFrame({
    'Feature': feature_cols,
    'High-Risk Value': [high_risk_student[f].values[0] for f in feature_cols],
    'High-Risk SHAP': individual_shap,
    'Low-Risk Value': [low_risk_student[f].values[0] for f in feature_cols],
    'Low-Risk SHAP': low_risk_shap
})

print(f"Student Comparison")
print("=" * 70)
print(f"High-Risk Student: P(Departed) = {high_risk_prob:.1%}")
print(f"Low-Risk Student:  P(Departed) = {low_risk_prob:.1%}")
print("\nKey Feature Differences:")
comparison[['Feature', 'High-Risk Value', 'Low-Risk Value']].head(10)

## 6. Summary

In this notebook, we covered:

### Key Concepts

1. **Early Stopping**:
   - Prevents overfitting by monitoring validation performance
   - Automatically finds optimal number of iterations
   - Supported natively in XGBoost, LightGBM, and CatBoost

2. **Cross-Validation**:
   - Use stratified K-fold for imbalanced classes
   - Combine with early stopping for robust estimates
   - Average best iterations across folds for final model

3. **Comprehensive Evaluation**:
   - Multiple metrics: accuracy, precision, recall, F1, ROC-AUC
   - ROC and Precision-Recall curves for discrimination
   - Calibration curves for probability reliability

4. **SHAP Interpretation**:
   - Global importance via mean absolute SHAP values
   - Dependence plots for feature effects
   - Individual explanations for stakeholder communication

### Summary Table

| Topic | Key Takeaway |
|:------|:-------------|
| Early Stopping | Set high n_estimators, let early stopping find optimal |
| Validation Split | Use 20-25% for early stopping |
| Cross-Validation | 5-fold stratified is standard |
| Calibration | Check Brier score and calibration curve |
| SHAP | Best method for explaining individual predictions |

### Next Steps

In the next notebook, we will focus on **hyperparameter tuning** to optimize model performance.

**Proceed to:** `4.5 Tune Boosted Model Hyperparameters`